In [1]:
using ITensors
using HDF5
using Random

In [2]:
fid = h5open("../mytestfile2.hdf5", "r")

e_rhf = read(fid, "e_rhf")
e_fci = read(fid, "e_fci")
h1e = read(fid, "h1e")
h2e = read(fid, "h2e")
nel = read(fid, "nel")
e_nuc = read(fid, "nuc")

close(fid)

println(e_rhf)
println(e_fci)
println(size(h1e))
println(size(h2e))
println(nel)
println(e_nuc)

-1.8837662820483878
-2.031756908333411
(8, 8)
(8, 8, 8, 8)
4
2.865078432261832


In [3]:
s = siteind("Fermion")

(dim=2|id=999|"Fermion,Site")

In [4]:
@show op("c†", s);

op("c†", s) = ITensor ord=2
Dim 1: (dim=2|id=999|"Fermion,Site")'
Dim 2: (dim=2|id=999|"Fermion,Site")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
 0.0  0.0
 1.0  0.0


In [5]:
function hf(i, nel)
    if i <= nel
        return 2
    else
        return 1
    end
end


hf (generic function with 1 method)

In [6]:
N_spt = size(h1e, 1)
N = 2*N_spt
sites = siteinds("Fermion",N)

# Choosing the site ordering:
ord = randperm(N_spt)
print(ord)

ampo = OpSum()

for p=1:N_spt
    for q=1:N_spt
        for σ=1:2
            ampo += h1e[ord[p],ord[q]],"c†",2*(p-1)+σ,"c",2*(q-1)+σ
        end
    end
end

for p=1:N_spt
    for q=1:N_spt
        for r=1:N_spt
            for s=1:N_spt
                for σ=1:2
                    for τ=1:2
                        ampo += 0.5*h2e[ord[p],ord[q],ord[r],ord[s]],"c†",2*(p-1)+σ,"c†",2*(r-1)+τ,"c",2*(s-1)+τ,"c",2*(q-1)+σ
                    end
                end
            end
        end
    end
end

println("Got OpSum!")

H = MPO(ampo,sites,cutoff=1E-6)

println("Got H!")

hf_occ = [hf((ord[cld(i,2)]*2-i%2), nel) for i=1:N]

println(hf_occ)
println(sites)

psi0 = MPS(sites,hf_occ)

println("Got psi0!")

E_hf = inner(psi0', H, psi0)

println("HF energy:")
println(E_hf+e_nuc)

sweeps = Sweeps(3) # number of sweeps is 5
maxdim!(sweeps,10,10,20) # gradually increase states kept
cutoff!(sweeps,1E-10) # desired truncation error

energy,psi = dmrg(H,psi0,sweeps)

println(energy+e_nuc)


[4, 3, 1, 7, 5, 6, 8, 2]Got OpSum!


LoadError: InterruptException:

In [7]:
sweeps = Sweeps(5) # number of sweeps is 5
maxdim!(sweeps,10,10,10,10,10) # gradually increase states kept
cutoff!(sweeps,1E-10) # desired truncation error

psi0 = randomMPS(sites,2)

energy,psi = dmrg(H,psi0,sweeps)

println(energy+e_nuc)



LoadError: UndefVarError: H not defined

In [8]:
dens = expect(psi,"N")
println("Particle number:")
println(sum(dens))

LoadError: UndefVarError: psi not defined

In [9]:
function swap_element(i,j,ip,jp)
    if ip==j && jp==i
        return 1
    else
        return 0
    end
end

function build_swap(sites, idx)
    swap = ITensor(sites[idx],sites[idx+1],sites[idx]',sites[idx+1]')
    for i=1:2
        for j=1:2
            for ip=1:2
                for jp=1:2
                    swap[i,j,ip,jp] = swap_element(i,j,ip,jp)
                end
            end
        end
    end
    
    return swap
    
end

build_swap (generic function with 1 method)

In [10]:
idx = 3
swap = build_swap(sites, idx)
println(swap)

ITensor ord=4
Dim 1: (dim=2|id=372|"Fermion,Site,n=3")
Dim 2: (dim=2|id=19|"Fermion,Site,n=4")
Dim 3: (dim=2|id=372|"Fermion,Site,n=3")'
Dim 4: (dim=2|id=19|"Fermion,Site,n=4")'
NDTensors.Dense{Int64, Vector{Int64}}
 2×2×2×2
[:, :, 1, 1] =
 1  0
 0  0

[:, :, 2, 1] =
 0  1
 0  0

[:, :, 1, 2] =
 0  0
 1  0

[:, :, 2, 2] =
 0  0
 0  1


In [11]:
orthogonalize!(psi,idx)

LoadError: UndefVarError: psi not defined

In [12]:
temp_tensor = (psi[idx] * psi[idx+1]) * swap
noprime!(temp_tensor)

LoadError: UndefVarError: psi not defined

In [13]:
temp_inds = uniqueinds(psi[idx],psi[idx+1])
U,S,V = svd(temp_tensor,temp_inds,cutoff=1E-8)
psi[idx] = U
psi[idx+1] = S*V

LoadError: UndefVarError: psi not defined

In [14]:
println(psi[idx])
#println(psi[idx+1])

LoadError: UndefVarError: psi not defined